# 01 · Exploration des données olympiques

Premier passage exploratoire pour comprendre les jeux de données et guider les étapes de préparation et de modélisation.

## Objectifs du notebook
- Inventorier les sources disponibles (résultats, médailles, athlètes).
- Vérifier la qualité des colonnes principales (types, pourcentages de valeurs manquantes).
- Réaliser des premiers graphiques descriptifs (répartition des médailles, pays dominants).
- Identifier des axes d'enrichissement pour les étapes suivantes.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid', context='talk')
np.random.seed(42)

In [ ]:
BASE_DIR = Path('..').resolve()
DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw' if (DATA_DIR / 'raw').exists() else DATA_DIR
REPORTS_FIG_DIR = BASE_DIR / 'reports' / 'figures'
REPORTS_FIG_DIR.mkdir(parents=True, exist_ok=True)

RAW_DIR

In [ ]:
results_df = pd.read_csv(RAW_DIR / 'olympic_results.csv')
medals_df = pd.read_csv(RAW_DIR / 'olympic_medals.csv')
athletes_df = pd.read_csv(RAW_DIR / 'olympic_athletes.csv')

results_df.shape, medals_df.shape, athletes_df.shape

## Aperçu rapide des tables
On inspecte quelques lignes et informations de structure pour détecter des besoins de nettoyage.

In [ ]:
results_df.head(3)

In [ ]:
results_df.isna().mean().sort_values(ascending=False).head(10)

In [ ]:
medals_df.head(3)

In [ ]:
athletes_df.sample(5, random_state=42)

## Répartition des médailles
On commence par regarder la distribution des types de médailles dans `olympic_results`.

In [ ]:
medal_counts = results_df['medal_type'].value_counts(dropna=False)
medal_counts

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.barplot(x=medal_counts.index.fillna('NO MEDAL'), y=medal_counts.values, palette='viridis', ax=ax)
ax.set_title('Distribution des médailles dans les résultats')
ax.set_xlabel('Type de médaille')
ax.set_ylabel("Nombre d'entrées")
plt.tight_layout()
fig.savefig(REPORTS_FIG_DIR / 'medal_distribution_results.png', dpi=120)
fig

## Pays les plus médaillés (données `olympic_medals`)
On agrège les médailles par pays pour repérer les dominants et comparer avec les objectifs (France et Top 25).

In [ ]:
top_countries = (medals_df.groupby('country_name')['medal_type']
                       .value_counts()
                       .rename('count')
                       .reset_index())
top25 = (top_countries.groupby('country_name')['count']
                        .sum()
                        .sort_values(ascending=False)
                        .head(25))
top25

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.barplot(x=top25.values, y=top25.index, palette='rocket', ax=ax)
ax.set_title('Top 25 des pays par nombre de médailles')
ax.set_xlabel('Nombre de médailles')
ax.set_ylabel('Pays')
plt.tight_layout()
fig.savefig(REPORTS_FIG_DIR / 'top25_medals.png', dpi=120)
fig

## Focus France
Vérifier la position de la France et préparer les futures prédictions spécifiques.

In [ ]:
france_medals = top_countries[top_countries['country_name'] == 'France']
france_medals

## Combinaison des jeux de données
Cette section prépare une version simplifiée qui permet de mesurer les doublons potentiels entre résultats et liste des médailles.

In [ ]:
merged_preview = (
    results_df[['discipline_title', 'event_title', 'slug_game', 'country_name', 'medal_type']]
    .merge(
        medals_df[['discipline_title', 'event_title', 'slug_game', 'country_name']],
        on=['discipline_title', 'event_title', 'slug_game', 'country_name'],
        how='left',
        indicator=True,
    )
)
merged_preview['_merge'].value_counts()

## Axes d'attention
- `olympic_results` contient des colonnes de listes (par équipes) qu'il faudra exploser lors de la préparation.
- Plusieurs colonnes ont des valeurs manquantes (notamment `athletes`, `medal_type`).
- Les jeux couvrent diverses éditions (variable `slug_game`), utile pour des agrégations temporelles.
- L'ajout de données socio-économiques pourra améliorer la future modélisation (PIB, population).

## Prochaines étapes
- Nettoyer et fusionner les datasets dans `02_preprocessing.ipynb`.
- Documenter les choix méthodologiques dans `docs/README_methodo.md`.
- Enregistrer les figures dans `reports/figures` (déjà sauvegardées automatiquement).